In [2]:
from datasets import load_dataset

dataset_name = "lamini/lamini_docs"
datasets = load_dataset(dataset_name)

In [3]:
data = []

for qa_pair in datasets['train']:
    _sentence = (qa_pair['question'] + ' ' + qa_pair['answer']).replace("\\n", " ")
    data.append(_sentence)

### Get top tf-idf words from the QA docs

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

def top_tfidf_words(sentences, top_n=100):
    # Initialize a TF-IDF Vectorizer
    vectorizer = TfidfVectorizer(stop_words='english')

    # Fit and transform the sentences
    tfidf_matrix = vectorizer.fit_transform(sentences)

    # Get feature names (words)
    feature_names = vectorizer.get_feature_names_out()

    # Sum TF-IDF for each term across all documents
    sums = tfidf_matrix.sum(axis=0)

    # Create a dataframe with words and their corresponding sums
    data = []
    for col, term in enumerate(feature_names):
        data.append((term, sums[0, col]))

    ranking = pd.DataFrame(data, columns=['term', 'rank'])
    ranking = ranking.sort_values('rank', ascending=False)

    return ranking

# Get top 100 TF-IDF words
top_words = top_tfidf_words(data)
print(top_words)


             term        rank
1848       lamini  103.728120
3253         text   54.182434
834          data   47.727913
2100        model   41.152211
142            ai   39.693581
...           ...         ...
3523     welcomes    0.066917
273     attending    0.066917
816     curiosity    0.066917
2385        plays    0.066917
888   departments    0.066917

[3574 rows x 2 columns]


In [10]:
top_words.iloc[0:40]

,term,rank
1848,lamini,103.728120
3253,text,54.182434
834,data,47.727913
2100,model,41.152211
142,ai,39.693581
3014,specific,39.275116
1937,llm,38.782491
1433,generate,38.024717
1855,language,36.952806
2103,models,35.657740


In [8]:
ww = ['chatbots',
 'hyperparameter',
 'hyperparameters',
 'lamini',
 'llamaindex',
 'llms',
 'tpus']
count = 0
for i, x in enumerate(top_words.iterrows()):
   if x[1][0] not in glove_words:
      count += 1
   if x[1][0] in ww:
     print(count, x[1][0], x[1][1])

/tmp/ipykernel_2991976/789917945.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if x[1][0] not in glove_words:
/tmp/ipykernel_2991976/789917945.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if x[1][0] in ww:
/tmp/ipykernel_2991976/789917945.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(count, x[1][0], x[1][1])


1 lamini 103.72812011895267
2 llms 16.911848466611534
6 chatbots 4.882678581053292
8 llamaindex 3.4261720020630055
10 hyperparameters 3.108621252031838
18 hyperparameter 2.011977616234558
78 tpus 0.33744743288350704


### Filter domains specific words (Remove GloVe words)

In [6]:
glove_words = []
with open("/nethome/ss651/DLT-Hallucination-Measurement/glove.6B.100d.txt", "r") as f:
    for line in f:
        word = line.split()[0]
        glove_words.append(word)

important_words = top_words["term"].tolist()

filtered_words = []

for word in important_words:
    if word not in glove_words:
        filtered_words.append(word)

filtered_words =  sorted(filtered_words)

In [12]:
filtered_words

['10mbps',
 '20gb',
 '23465125488609597',
 '3627493468056o869069738746723563537579456800',
 '37k',
 '410m',
 '70k',
 '800k',
 '84092384972847832590458',
 '__call__',
 '__call___',
 '__callable__',
 '__init__',
 'add_data',
 'add_improve_statement',
 'add_improve_statements',
 'add_metric',
 'add_model',
 'animal_stories',
 'backoff',
 'bad_examples',
 'basemodel',
 'basemodels',
 'byeeee',
 'camelinae',
 'cancel_job',
 'changelog',
 'chatbot',
 'chatbots',
 'chatgpt',
 'check_job_status',
 'circular_operation',
 'cohere_throughput',
 'compare_equal_metric',
 'composable',
 'configure_llama',
 'databricks',
 'dataframe',
 'datasetbalancer',
 'debiasing',
 'deduped',
 'detaileddescriptors',
 'dhqdnoerijtoigjro',
 'diamos',
 'distributeddataparallel',
 'eachadea',
 'edit_config',
 'eleutherai',
 'erqiujlkcmabhsvandlkfhpghl',
 'error_handling',
 'explainability',
 'feedbackoperation',
 'filter_fn',
 'fostes',
 'fucntion',
 'full_balance_dataset',
 'gen_queue_batch',
 'gen_submit_training_j

#### Compare the domain specific words with NERs

In [7]:
import pandas as pd

df = pd.read_csv("../results/train_ners.csv")
ner_list = list(df["NERS"].dropna())

ners = []
for ner in ner_list:
    ners.extend(ner.lower().replace("`","").split(", "))

filtered_ners = []
for ner in ners:
    if ner not in glove_words:
        filtered_ners.append(ner)

In [14]:
import csv

with open('../results/important_words.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for _word in filtered_words:
        writer.writerow([_word])

### Extract Lamini Specific function calls from the domain specific words

In [7]:
import re

document_content = ' '.join(data)

# Regex pattern for function calls
pattern = r'\b[a-zA-Z0-9]+\w*_+\w+?(?<!\.py)\b|\b[a-zA-Z0-9]+\w+\(\)(?<!\.py\b)'

entities = re.findall(pattern, document_content)
entities = sorted(list(set(entities)))


In [19]:
for x in data:
    if  'test_output_str' in x:
        print(x)

Is it possible to customize the level of specificity in the generated output? Yes, it is possible to customize the level of specificity in the generated output. This can be achieved by adjusting the input parameters and output type in the LLM Engine function, as demonstrated in the "TestOutputStr" class in the "test_output_str.py" file. By defining specific input parameters and output types, the generated output can be tailored to meet the desired level of specificity.


In [11]:
filtered_entities = [
 'add_data',
 'add_improve_statements',
 'add_metric',
 'add_model',
 'bad_examples',
 'cancel_job',
 'check_job_status',
 'circular_operation',
 'compare_equal_metric',
 'configure_llama',
 'edit_config',
 'error_handling',
 'filter_fn',
 'full_balance_dataset',
 'gen_queue_batch',
 'gen_submit_training_job',
 'get_job_results',
 'get_response',
 'good_examples',
 'improve()',
 'is_peft_model',
 'length_penalty',
 'llm()',
 'make_discriminator',
 'make_questions',
 'max_retries',
 'max_tokens',
 'model_name',
 'parse_response',
 'repetition_penalty',
 'run_all',
 'sample()',
 'stochastic_balance_dataset',
 'submit_job',
 'test_cache',
 'test_output_str',
 'test_parallel_complex',
 'test_parallel_simple',
 'value_to_dict',
 'write_story']

In [12]:
import csv
file_path = "../results/filtered_entities.csv"

with open(file_path, 'w', newline='') as file:
   writer = csv.writer(file)
   for _word in filtered_entities:
      writer.writerow([_word])

### Extract Context for the filtered words

In [33]:
context = []

for func in filtered_entities:
    _context = []
    for qa in data:
        if func in qa:
            _context.append(qa)
    context.append([func, ' '.join(_context)])

import pickle as pkl
pkl.dump(context, open("context.pkl", "wb"))

In [34]:
import csv
file_path = 'context.csv'

# Writing to a CSV file
with open(file_path, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Entity", "Context"])
    writer.writerows(context)


In [46]:
summary = pkl.load(open("summary.pkl", "rb"))

file_path = 'summary.csv'

# Writing to a CSV file
with open(file_path, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Entity", "Summary"])
    writer.writerows(summary)
